# The Cutting Stock Problem
You are given a product with a standard length of `totalLength`.
This product (e.g., a steel coil) can be cut down to smaller lengths (`lengths`).
For these smaller lengths, you face deterministic demand (`demand`).

The task is to determine possible patterns of cutting the standard length product into pieces with smaller lengths so that
- the total scrap / waste is minimized or
- the number of standard length pieces used is minimized

Smaller length pieces that exceed demand can be stored, but there is only a limited storage capacity (`capacity`).

Note that the number of smaller lengths can be easily extended to > 2 (adjusting `lengths` and `demand`). 

In [1]:
# parameters
totalLength = 210
lengths = [45, 27]
demand = [200, 3000]  # demand for each length
capacity = 160  # capacity of storage for waste / remainings

In [2]:
# determine maximum amount of pieces for each smaller length per standard length
max_ = {}
num = len(lengths)
for i in range(0, num):
    max_[i] = int(totalLength/lengths[i])
    print(f"Maximum number of pieces for item {i}: {max_[i]}")

Maximum number of pieces for item 0: 4
Maximum number of pieces for item 1: 7


### Patterns
In a first step, we identify all possible patterns (i.e., combinations of cutting the standard length product into smaller length products).
For this, we calculate the total number of possible combinations / patterns (`numCombinations`) and map each of these patterns to a number between `0` and `numCombinations`.

In [3]:
multipliers = {}
for i in range(0, num):
    multipliers[i] = 1
for i in range(0, num-1):
    for j in range(i+1, num):
        multipliers[i] *= (max_[j]+1)

numCombinations = 1
for i in range(0, num):
    numCombinations *= max_[i]+1

print(f"Number of possible patterns: {numCombinations}")
    
patterns = []
for i in range(0, numCombinations):
    comb = ()
    rem = i
    for j in range(0, num):
        temp = int(rem/multipliers[j])
        rem = i % multipliers[j]
        comb = comb + (temp,)
    len_ = 0
    for i in range(0, num):
        len_ += comb[i]*lengths[i]
    patterns.append({
        "pattern": comb,
        "length": len_,
        "scrap": totalLength - len_
    })
print()
print("pattern\t | length\t | scrap")
for i in range(0, len(patterns)):
    p = patterns[i]
    print(f"{p['pattern']}\t | {p['length']}\t\t | {p['scrap']}")

Number of possible patterns: 40

pattern	 | length	 | scrap
(0, 0)	 | 0		 | 210
(0, 1)	 | 27		 | 183
(0, 2)	 | 54		 | 156
(0, 3)	 | 81		 | 129
(0, 4)	 | 108		 | 102
(0, 5)	 | 135		 | 75
(0, 6)	 | 162		 | 48
(0, 7)	 | 189		 | 21
(1, 0)	 | 45		 | 165
(1, 1)	 | 72		 | 138
(1, 2)	 | 99		 | 111
(1, 3)	 | 126		 | 84
(1, 4)	 | 153		 | 57
(1, 5)	 | 180		 | 30
(1, 6)	 | 207		 | 3
(1, 7)	 | 234		 | -24
(2, 0)	 | 90		 | 120
(2, 1)	 | 117		 | 93
(2, 2)	 | 144		 | 66
(2, 3)	 | 171		 | 39
(2, 4)	 | 198		 | 12
(2, 5)	 | 225		 | -15
(2, 6)	 | 252		 | -42
(2, 7)	 | 279		 | -69
(3, 0)	 | 135		 | 75
(3, 1)	 | 162		 | 48
(3, 2)	 | 189		 | 21
(3, 3)	 | 216		 | -6
(3, 4)	 | 243		 | -33
(3, 5)	 | 270		 | -60
(3, 6)	 | 297		 | -87
(3, 7)	 | 324		 | -114
(4, 0)	 | 180		 | 30
(4, 1)	 | 207		 | 3
(4, 2)	 | 234		 | -24
(4, 3)	 | 261		 | -51
(4, 4)	 | 288		 | -78
(4, 5)	 | 315		 | -105
(4, 6)	 | 342		 | -132
(4, 7)	 | 369		 | -159


In the next step, we identify valid patterns (total length does not exceed standard product length `totalLength`) and undominated patterns (it is not possible to cut an additional smaller length piece).

In [4]:
patterns_select = []
for p in patterns:
    p["valid"] = (p["length"] <= totalLength)
    p["undominated"] = True
    for i in range(0, num):
        p["undominated"] = p["undominated"] and (p["scrap"] < lengths[i])
    if p["valid"] and p["undominated"]:
        patterns_select.append(p)

print(f"Number of valid and undominated patterns: {len(patterns_select)}")
print()
print("pattern\t | length\t | scrap")
for i in range(0, len(patterns_select)):
    p = patterns_select[i]
    print(f"{p['pattern']}\t | {p['length']}\t\t | {p['scrap']}")

Number of valid and undominated patterns: 5

pattern	 | length	 | scrap
(0, 7)	 | 189		 | 21
(1, 6)	 | 207		 | 3
(2, 4)	 | 198		 | 12
(3, 2)	 | 189		 | 21
(4, 1)	 | 207		 | 3


### Optimization

In [28]:
import gurobipy as gp
from gurobipy import GRB

model = gp.Model()
model.Params.LogToConsole = 0

x = {}
for i in range(len(patterns_select)):
    x[i] = model.addVar(vtype=GRB.INTEGER, lb = 0)
y = {}
for i in range(num):
    y[i] = model.addVar(vtype=GRB.CONTINUOUS, lb = 0)

for i in range(num):
    model.addConstr(
        gp.quicksum(x[j]*patterns_select[j]['pattern'][i] for j in range(len(patterns_select))) == demand[i] + y[i]
    )

model.addConstr(
    gp.quicksum(y[i] for i in range(num)) <= capacity
)

obj = {
    "minScrap": gp.quicksum(x[i]*patterns_select[i]['scrap'] for i in range(len(patterns_select))),
    "minRolls": gp.quicksum(x[i] for i in range(len(patterns_select)))
}

for o in obj.keys():
    print(f"+++++ {o} +++++")
    model.setObjective(obj[o], GRB.MINIMIZE)
    model.optimize()

    print("pattern\t | # of times in optimal solution")
    for i in range(0, len(patterns_select)):
        print(f"{patterns_select[i]['pattern']}\t | {x[i].x}")

    print()
    print("length\t | demand\t | sum\t\t | leftovers")
    for i in range(0, num):
        total = sum([x[p].x*patterns_select[p]["pattern"][i] for p in range(0, len(patterns_select))])
        print(f"{lengths[i]}\t | {demand[i]}\t\t | {total}\t | {total - demand[i]}")
    print()

    scrap = sum([x[p].x*patterns_select[p]["scrap"] for p in range(0, len(patterns_select))])
    print(f"scrap = {scrap}")

    numRolls = sum([x[p].x for p in range(0, len(patterns_select))])
    print(f"numRolls = {numRolls}")
    print()

+++++ minScrap +++++
pattern	 | # of times in optimal solution
(0, 7)	 | 120.0
(1, 6)	 | 360.0
(2, 4)	 | -0.0
(3, 2)	 | -0.0
(4, 1)	 | -0.0

length	 | demand	 | sum		 | leftovers
45	 | 200		 | 360.0	 | 160.0
27	 | 3000		 | 3000.0	 | 0.0

scrap = 3600.0
numRolls = 480.0

+++++ minRolls +++++
pattern	 | # of times in optimal solution
(0, 7)	 | 257.0
(1, 6)	 | 200.0
(2, 4)	 | 0.0
(3, 2)	 | 0.0
(4, 1)	 | 1.0

length	 | demand	 | sum		 | leftovers
45	 | 200		 | 204.0	 | 4.0
27	 | 3000		 | 3000.0	 | 0.0

scrap = 6000.0
numRolls = 458.0

